# Restaurant Recommendation

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Data Collection

In [2]:
dt = pd.read_csv(r"C:\Users\Seelam Nagasri\OneDrive\MLDataset.csv")
pd.set_option('display.max_columns', None)
dt

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",1100,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,1200,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",4000,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",1500,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",1500,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9546,5915730,Naml۱ Gurme,208,��stanbul,"Kemanke�� Karamustafa Pa��a Mahallesi, R۱ht۱m ...",Karak�_y,"Karak�_y, ��stanbul",28.977392,41.022793,Turkish,80,Turkish Lira(TL),No,No,No,No,3,4.1,Green,Very Good,788
9547,5908749,Ceviz A��ac۱,208,��stanbul,"Ko��uyolu Mahallesi, Muhittin ��st�_nda�� Cadd...",Ko��uyolu,"Ko��uyolu, ��stanbul",29.041297,41.009847,"World Cuisine, Patisserie, Cafe",105,Turkish Lira(TL),No,No,No,No,3,4.2,Green,Very Good,1034
9548,5915807,Huqqa,208,��stanbul,"Kuru�_e��me Mahallesi, Muallim Naci Caddesi, N...",Kuru�_e��me,"Kuru�_e��me, ��stanbul",29.034640,41.055817,"Italian, World Cuisine",170,Turkish Lira(TL),No,No,No,No,4,3.7,Yellow,Good,661
9549,5916112,A���k Kahve,208,��stanbul,"Kuru�_e��me Mahallesi, Muallim Naci Caddesi, N...",Kuru�_e��me,"Kuru�_e��me, ��stanbul",29.036019,41.057979,Restaurant Cafe,120,Turkish Lira(TL),No,No,No,No,4,4.0,Green,Very Good,901


# Data Preprocessing
#### Handling missing valued and Encoding categorical variabless

In [3]:
enc = dt.drop_duplicates(subset=['Restaurant Name'],keep='first')
encoder= OneHotEncoder()
encoded_features =encoder.fit_transform(dt[['City','Cuisines', 'Rating text',]])

In [4]:
feature_matrix = pd.DataFrame(encoded_features.toarray(), columns=encoder.get_feature_names_out())

In [5]:
similarity_matrix = cosine_similarity(feature_matrix)

In [6]:
similarity_df = pd.DataFrame(similarity_matrix, index=dt['Restaurant Name'], columns=dt['Restaurant Name'])

###  Criteria 1 for Restaurant Recommendations
#### User Preferences of Cuicines, City, Ratings

In [7]:
def recommend_restaurants(restaurant_name, num_recommendations=5):
    if not isinstance(restaurant_name, list):
        print("Restaurant not found!")
        return 0
    else:
        return similarity_df[restaurant_name].sum(axis=1).sort_values(ascending=False).iloc[1:num_recommendations+1].index.tolist()

In [8]:
import random
print(random.choice(pd.unique(dt['Restaurant Name']).tolist()))

Amritsari Chaap Corner


In [9]:
user_preferences = {
    
    'Cuisines':['Seafood', 'Filipino, Asian', 'European'],
    'City': 'Pasay City',
    #'Price range': '',
    'Rating' : 2,
    
}
print(user_preferences)

{'Cuisines': ['Seafood', 'Filipino, Asian', 'European'], 'City': 'Pasay City', 'Rating': 2}


 Chooses Random fields from dataset

In [10]:
user_preferences = {
    
    'Cuisines': random.choice(pd.unique(dt['Cuisines']).tolist()), #['Seafood', 'Filipino, Asian', 'European'],
    'City': random.choice(pd.unique(dt['City']).tolist()),         #'Pasay City',
    'Rating' : 2,
    
} 
print(user_preferences)

{'Cuisines': 'Seafood', 'City': 'Aurangabad', 'Rating': 2}


In [11]:
matching_restaurants = dt[
    (dt['City'] == user_preferences['City']) &
    (dt['Cuisines'].str.contains('|'.join(user_preferences['Cuisines']), case=False, na=False)) &
    (dt['Aggregate rating'] > user_preferences['Rating'])
]
view_res = matching_restaurants[['Restaurant ID','Restaurant Name','Address','Cuisines','Average Cost for two','Currency','Rating text','Aggregate rating',]]
view_res

,Restaurant ID,Restaurant Name,Address,Cuisines,Average Cost for two,Currency,Rating text,Aggregate rating
706,2500023,Kream N Krunch,"2, Near Akashwani Circle, Mahesh Nagar, Jalna ...","North Indian, Chinese, Continental, Italian",800,Indian Rupees(Rs.),Good,3.6
707,2500076,Balbeer's Kitchen & Bar,"Shendra, Near Cambridge School, Jalna Road, Ch...","Italian, Continental, Chinese, North Indian",850,Indian Rupees(Rs.),Average,3.3
708,2500054,Angeethi Restaurant,"Seven Hill Flyover, Jalna Road, Vidya Nagar, C...","North Indian, Chinese",600,Indian Rupees(Rs.),Average,3.3
709,2500134,Domino's Pizza,"8, Upper Ground Floor, City Pride Commercial C...",Pizza,700,Indian Rupees(Rs.),Average,3.1
710,2500069,Hotel Laadli,"Goodwill Complex, Opposite Cidco Busstand, N-2...","North Indian, South Indian",350,Indian Rupees(Rs.),Average,3.4
711,2500256,Indiana Veg Restaurant,"Plot 2, Near High Court, N-3, CIDCO, Aurangabad","South Indian, North Indian, Chinese, Fast Food",450,Indian Rupees(Rs.),Average,3.4
712,2500024,Madhuban Restaurant - Welcome Hotel Rama Inter...,"Welcome Hotel Rama International, Opposite Hig...","North Indian, Chinese, Continental, Italian, M...",1800,Indian Rupees(Rs.),Average,3.4
713,2500075,Mauj Restaurant,"31, Kailash Arcade, Opposite Bank of Baroda, C...","Italian, Fast Food, North Indian, Chinese",500,Indian Rupees(Rs.),Average,3.3
714,2500029,Naivedya,"Sakal Office, Jalna Road, CIDCO, Aurangabad",North Indian,500,Indian Rupees(Rs.),Average,3.4
715,2500079,d' Curry House,"Hotel Green Olive, Near Baba Petrol Pump, Nira...","Continental, Chinese, Biryani, North Indian",1000,Indian Rupees(Rs.),Good,3.6


 Sometimes you didn't get enough records because random values won't match, generate random values again 

In [12]:
                                                            #same preferences which use initially
                                                        # Find all restaurants matching these preferences
user_preferences = {'Cuisines': 'Asian, Chinese, Vegetarian', 'City': 'Chandigarh', 'Rating': 2}
matching_restaurants = dt[
    (dt['City'] == user_preferences['City']) &
    (dt['Cuisines'].str.contains('|'.join(user_preferences['Cuisines']), case=False, na=False)) &
    (dt['Aggregate rating'] > user_preferences['Rating'])

]

# Recommend similar ones based on these preferences
recommendations = recommend_restaurants(matching_restaurants['Restaurant Name'].tolist(), num_recommendations=6)
recommendations = list(set(recommendations))
print("Recommended Restaurants:", recommendations)

Recommended Restaurants: ['Barbeque Nation', 'The Culinary Pitaara', 'Aromas of Pind']


In [13]:
dt[dt['Restaurant Name'].isin(recommendations)].sort_values(by='Aggregate rating',ascending=False).head(5)[['Restaurant ID','Restaurant Name','Address','Cuisines','Average Cost for two','Currency','Rating text','Aggregate rating']]

,Restaurant ID,Restaurant Name,Address,Cuisines,Average Cost for two,Currency,Rating text,Aggregate rating
9262,2800856,Barbeque Nation,"1st Floor, ATR Towers, Harbour Park Road, Pan...","North Indian, Chinese, Mediterranean",1600,Indian Rupees(Rs.),Excellent,4.9
2536,3300958,Barbeque Nation,"2nd Floor, Eternity Mall, Amravati Road, Sitab...","North Indian, European, Mediterranean",1600,Indian Rupees(Rs.),Excellent,4.9
2414,20842,Barbeque Nation,"K1, RDB Boulevard, Block EP & GP, Sector 5, Sa...","North Indian, Chinese",1600,Indian Rupees(Rs.),Excellent,4.9
2289,2100702,Barbeque Nation,"2nd Floor, Adityam Building, Ulubari, Guwahati",North Indian,1500,Indian Rupees(Rs.),Excellent,4.9
2409,25570,Barbeque Nation,"24, 1st Floor, Park Center Building, Park Stre...","North Indian, Chinese",1600,Indian Rupees(Rs.),Excellent,4.9
